<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dcef4a8c5caf97ca0890a2293d339ee8c60fa003fcae6ac7d6da9696249a94ab
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-29 13:47:49
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -86.96 K (-0.62%)
Current PnL: -26.74 L (-17.27%)
CY Booked + Current PnL: -12.45 L (-8.04%)
-------------------
Total profit:  1.49 L
Total loss:  -28.23 L
-------------------
Total Booked + Current PnL: 14.53 L (11.37%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.18%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.37 L (65.12%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.09%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.78,-19.40,24.18,0.09,19641.0,-19550.0,81227.0,88.47,38.0,M-SC,2.92,253.0,-1.00,0.58,6.71,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.57,8.66,10.15,19.69,21148.0,16608.0,208350.0,-1.91,54.0,X-MC,2.40,78.0,0.79,1.49,16.05,XY25,NTT,AC
52,NESTLEIND,1377.00,0.74,13.02,7.41,21.40,22229.0,34562.0,299988.0,4.80,65.0,X-LC,5.53,12.0,1.55,2.15,20.17,XY25,NTT,FMCG
50,MASFIN,398.61,-1.19,-4.44,27.72,22.05,25954.0,-4350.0,93630.0,-17.80,47.0,H-SC,6.51,164.0,-0.17,0.67,36.10,XR,ATH,FINANCE
43,ITC,452.00,-0.46,-1.40,12.35,10.78,29264.0,-3363.0,236955.0,-41.15,47.0,X-LC,2.05,5.0,-0.11,1.70,3.76,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,3.54,-38.20,118.88,35.27,83425.0,-43373.0,70176.0,-51.71,54.0,L-SC,8.78,270.0,-0.52,0.50,71.76,XR,NTT,HOTELS
57,RAJOOENG,143.10,1.89,-38.59,127.32,39.61,80148.0,-39550.0,62950.0,-58.53,34.0,H-SC,31.78,135.0,-0.49,0.45,1.89,AR,ATH,MISC
44,JCHAC,2282.00,1.47,-34.82,53.40,-0.01,44485.0,-44499.0,83306.0,16943.48,42.0,M-SC,12.53,233.0,-1.00,0.60,6.62,OX40N,NTT,AC
54,PGHH,17907.65,1.22,-1.78,36.62,34.19,76802.0,-3792.0,209728.0,-30.90,58.0,X-MC,4.76,57.0,-0.05,1.50,4.35,X40,ATH,FMCG
5,ASIANPAINT,3460.25,0.90,-3.14,24.86,20.95,60625.0,-7902.0,243866.0,-4.15,47.0,X-LC,14.83,31.0,-0.13,1.75,31.38,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
69,SURYODAY,216.00,-3.25,-20.91,55.56,23.03,78688.0,-37444.0,141627.0,53.51,50.0,H-SC,6.85,167.0,-0.48,1.01,40.52,XR,NTT,BANKS
22,DIXON,18931.72,-3.20,-20.53,58.64,26.07,90975.0,-40079.0,155142.0,-61.38,25.0,X-MC,14.01,56.0,-0.44,1.11,0.00,X40N,ATH,IT
25,FINCABLES,1641.55,-2.95,-5.70,120.48,107.91,167746.0,-8413.0,139231.0,-23.39,42.0,M-SC,11.68,216.0,-0.05,1.00,3.81,OX40N,ATH,CABLES
13,BSOFT,831.70,-2.89,-20.99,90.84,50.79,100950.0,-29524.0,111129.0,1.28,58.0,H-SC,9.96,151.0,-0.29,0.80,29.39,XR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.07,-0.89,83.3,81.67,77450.0,-833.0,92977.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.05,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.95,-5.70,120.48,107.91,167746.0,-8413.0,139231.0,-23.39,42.0,M-SC,11.68,216.0,-0.05,1.00,3.81,OX40N,ATH,CABLES
77,TTKPRESTIG,770.00,-0.78,-19.40,24.18,0.09,19641.0,-19550.0,81227.0,88.47,38.0,M-SC,2.92,253.0,-1.00,0.58,6.71,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.14,-20.02,25.06,0.02,34987.0,-34947.0,139612.0,113.17,33.0,M-SC,9.81,234.0,-1.00,1.00,21.44,OX40N,NTT,PAINTS
47,KANSAINER,340.00,-1.10,-21.64,44.80,13.47,94671.0,-58347.0,211320.0,-67.73,54.0,H-SC,6.82,158.0,-0.62,1.51,7.45,XY24,NTT,PAINTS
66,SIS,528.00,-1.39,-23.13,57.75,21.26,49288.0,-25684.0,85348.0,2015.03,54.0,H-SC,4.78,166.0,-0.52,0.61,15.31,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.48,8.7,58.48,72.27,107710.0,14734.0,184182.0,-6.53,60.0,M-LC,4.45,99.0,0.14,1.32,15.41,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.48,8.70,58.48,72.27,107710.0,14734.0,184182.0,-6.53,60.0,M-LC,4.45,99.0,0.14,1.32,15.41,XR,NTT,IT
6,ASIANTILES,137.00,-2.07,-0.89,83.30,81.67,77450.0,-833.0,92977.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.05,XR,NTT,CERAMICS
38,INDIAMART,4810.62,-0.56,-5.30,118.29,106.72,138164.0,-6535.0,116801.0,-53.11,37.0,H-SC,5.04,139.0,-0.05,0.84,16.84,AR,ATH,MISC
25,FINCABLES,1641.55,-2.95,-5.70,120.48,107.91,167746.0,-8413.0,139231.0,-23.39,42.0,M-SC,11.68,216.0,-0.05,1.00,3.81,OX40N,ATH,CABLES
1,ABB,7934.00,-0.06,-4.96,53.17,45.57,132202.0,-12979.0,248640.0,-38.95,51.0,H-MC,4.54,120.0,-0.10,1.78,5.80,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-3.20,-20.53,58.64,26.07,90975.0,-40079.0,155142.0,-61.38,25.0,X-MC,14.01,56.0,-0.44,1.11,0.00,X40N,ATH,IT
9,BAJAJHFL,181.50,-1.12,-19.43,92.96,55.47,182221.0,-47265.0,196021.0,-28.68,26.0,X-MC,16.54,64.0,-0.26,1.40,0.44,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.03,-18.97,76.87,43.32,101453.0,-30892.0,131980.0,-30.40,29.0,X-SC,6.59,90.0,-0.30,0.95,1.34,X40N,ATH,ELECTRICAL
8,AWL,485.00,-0.55,-25.42,105.81,53.50,238121.0,-76705.0,225046.0,-63.88,30.0,X-MC,6.84,58.0,-0.32,1.61,0.00,XY24,NTT,FMCG
3,ACC,3906.00,-0.60,-27.51,126.45,64.15,218114.0,-65461.0,172490.0,-57.62,31.0,X-SC,4.81,82.0,-0.30,1.24,0.00,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.56,-2.89,24.75,21.14,54640.0,-6573.0,220766.0,-9.80,47.0,X-MC,1.21,74.0,-0.12,1.58,22.60,XY24,NTT,PAINTS
78,UNITDSPR,1644.0,-0.38,4.53,15.54,20.77,38032.0,10595.0,244739.0,-4.59,49.0,X-MC,1.71,62.0,0.28,1.75,11.76,X40N,NTT,BREWERIES
15,CAMS,950.0,-0.68,-2.56,27.74,24.48,68080.0,-6435.0,245421.0,-81.32,42.0,X-SC,1.87,86.0,-0.09,1.76,19.99,X40N,NTT,MISC
65,SIEMENS,4671.5,-1.09,-17.59,52.31,25.51,80223.0,-32735.0,153360.0,-5.83,39.0,H-LC,1.96,49.0,-0.41,1.10,13.01,AR,ATH,ELECTRICAL
43,ITC,452.0,-0.46,-1.40,12.35,10.78,29264.0,-3363.0,236955.0,-41.15,47.0,X-LC,2.05,5.0,-0.11,1.70,3.76,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.60,-27.51,126.45,64.15,218114.0,-65461.0,172490.0,-57.62,31.0,X-SC,4.81,82.0,-0.30,1.24,0.00,XY24,BTT,CEMENT
8,AWL,485.00,-0.55,-25.42,105.81,53.50,238121.0,-76705.0,225046.0,-63.88,30.0,X-MC,6.84,58.0,-0.32,1.61,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-3.20,-20.53,58.64,26.07,90975.0,-40079.0,155142.0,-61.38,25.0,X-MC,14.01,56.0,-0.44,1.11,0.00,X40N,ATH,IT
19,COLPAL,3726.84,-0.87,-21.43,80.10,41.51,165751.0,-56435.0,206930.0,-6.53,33.0,X-MC,8.16,61.0,-0.34,1.48,0.00,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-1.12,-39.36,122.98,35.20,95949.0,-50650.0,78020.0,2.24,33.0,X-SC,17.26,92.0,-0.53,0.56,0.07,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.16,-29.30,102.05,42.84,46899.0,-19049.0,45957.0,-53.72,46.0,X-SC,33.01,83.0,-0.41,0.33,3.97,XY24,NTT,MISC
59,RELAXO,1176.00,-0.76,-48.18,193.89,52.31,145860.0,-69932.0,75228.0,-44.83,40.0,X-SC,7.31,91.0,-0.48,0.54,0.70,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.12,-39.36,122.98,35.20,95949.0,-50650.0,78020.0,2.24,33.0,X-SC,17.26,92.0,-0.53,0.56,0.07,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.45,-3.31,28.32,24.08,34762.0,-4202.0,122748.0,-11.10,41.0,X-SC,6.97,89.0,-0.12,0.88,15.66,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.03,-18.97,76.87,43.32,101453.0,-30892.0,131980.0,-30.40,29.0,X-SC,6.59,90.0,-0.30,0.95,1.34,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.81,-10.54,34.93,20.71,118191.0,-39873.0,338364.0,-22.98,57.0,X-LC,3.00,1.0,-0.34,2.42,13.06,X40,ATH,IT
41,INFY,1972.00,-0.78,11.86,20.01,34.24,70035.0,37114.0,350002.0,-12.78,61.0,X-LC,6.73,2.0,0.53,2.51,21.13,X40,NTT,IT
75,TMPV,600.00,0.01,-26.45,67.20,22.98,111651.0,-59750.0,166148.0,-24.30,49.0,X-LC,6.12,3.0,-0.54,1.19,4.50,XY24,NTT,AUTO
81,VBL,671.64,0.29,-2.18,38.70,35.67,119564.0,-6890.0,308952.0,-13.74,58.0,X-LC,3.21,4.0,-0.06,2.21,11.37,X40N,ATH,FMCG
43,ITC,452.00,-0.46,-1.40,12.35,10.78,29264.0,-3363.0,236955.0,-41.15,47.0,X-LC,2.05,5.0,-0.11,1.70,3.76,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.11,-37.95,114.18,32.89,54696.0,-29304.0,47903.0,-691.63,50.0,L-MC,6.77,259.0,-0.54,0.34,31.92,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-2.07,-0.89,83.30,81.67,77450.0,-833.0,92977.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.05,XR,NTT,CERAMICS
13,BSOFT,831.70,-2.89,-20.99,90.84,50.79,100950.0,-29524.0,111129.0,1.28,58.0,H-SC,9.96,151.0,-0.29,0.80,29.39,XR,ATH,IT
17,COALINDIA,484.83,-0.15,-1.25,20.74,19.23,30814.0,-1886.0,148574.0,15.50,73.0,L-LC,3.12,179.0,-0.06,1.06,19.75,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.07,-0.89,83.30,81.67,77450.0,-833.0,92977.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.05,XR,NTT,CERAMICS
13,BSOFT,831.7,-2.89,-20.99,90.84,50.79,100950.0,-29524.0,111129.0,1.28,58.0,H-SC,9.96,151.0,-0.29,0.80,29.39,XR,ATH,IT
74,TITAGARH,1548.0,0.74,-14.85,71.16,45.74,146734.0,-35974.0,206203.0,13.45,71.0,H-SC,5.83,174.0,-0.25,1.48,34.11,XY24,NTT,ENGINEERING
56,RAJESHEXPO,518.0,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
84,WIPRO,420.0,-0.48,8.70,58.48,72.27,107710.0,14734.0,184182.0,-6.53,60.0,M-LC,4.45,99.0,0.14,1.32,15.41,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.45
1,25,45.03
2,50,76.69


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.94
MC    30.77
LC    24.33
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.91
X40      23.14
X40N     12.87
XR        9.80
XY25      9.22
AR        8.84
OX40N     7.53
SR        0.99
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.19
H-SC    23.83
X-LC    20.85
M-SC    11.49
X-SC     8.07
H-MC     4.84
L-SC     1.55
M-MC     1.40
M-LC     1.32
H-LC     1.10
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.43,-7.20,41.67
IT,12.93,-16.46,75.01
FINANCE,9.62,-18.51,70.04
MISC,7.18,-29.78,82.87
ELECTRICAL,5.96,-12.91,54.27
PAINTS,5.84,-13.21,30.03
INSURANCE,4.71,-3.18,38.48
PHARMA,4.06,-3.65,36.27
AUTO,2.84,-32.29,78.15


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3164488.0,21
AR,1339622.0,10
XR,1294154.0,13
X40,993011.0,14
X40N,888647.0,9
OX40N,740125.0,10
XY25,353289.0,6
SR,282601.0,2
MH,81465.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3653299.0,25
M-SC,1456872.0,15
X-MC,1446740.0,16
X-LC,815232.0,11
X-SC,794784.0,8
H-MC,402380.0,3
L-SC,244578.0,3
M-LC,107710.0,1
H-LC,80223.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1251866.0      6
           AR         914400.0      5
M-SC       XY24       824309.0      6
H-SC       XR         792882.0      7
X-MC       X40        490664.0      7
           XY24       411856.0      3
           X40N       357321.0      4
H-SC       OX40N      330085.0      4
M-SC       OX40N      326337.0      5
X-LC       X40        322731.0      5
X-SC       X40N       315393.0      3
           XY24       299775.0      3
H-SC       SR         282601.0      2
H-MC       AR         216764.0      2
X-LC       X40N       215933.0      2
           XY24       191066.0      2
X-MC       XY25       186899.0      2
H-MC       XY24       185616.0      1
X-SC       X40        179616.0      2
M-SC       XR         177991.0      2
L-SC       XR         160875.0      2
M-SC       AR         128235.0      2
M-LC       XR         107710.0      1
X-LC       XY25        85502.0      2
L-SC       OX40N       83703.0      1
H-SC       MH          81465.0      1
H-LC       AR          80223.0      1
L-MC       XR          54696.0      1
M-MC       XY25        50074.0      1
L-LC       XY25        30814.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
